# Package

In [1]:
#Basic
import re
import pandas as pd
import numpy as np
import json
import torch
from transformers import AutoTokenizer, ElectraModel
from tqdm import tqdm

#preprocessing
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from collections import Counter

#model
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import LayerNormalization
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 원본 정제 데이터 로드
with open("/content/drive/MyDrive/cleaned_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [4]:
original = pd.DataFrame(data).copy()

In [5]:
original.head()

,text,disasterType,urgencyLevel
0,"예, 엊그저, 엊그저께 코로나 주사 맞앗는데. | 몸이 좀 가슴이 안좋다네요 안면이...",구급 - 질병(중증 외),1
1,"아, 제가 9일날, 월요일날. | 화이자 백신을 맞았거든요. | 근데. | 지금, ...",구급 - 질병(중증 외),1
2,"예, 여보세요? | 저, 허리가 너무 아파서 그러는데. | 어떡해야. | 저, 아까...",구급 - 질병(중증 외),1
3,"예. | 네, 수고하십니다. 여기, 그, 월산동, 그, 새론 주유소인데요. | 새론...",구급 - 질병(중증 외),0
4,"여보세요? | 여보세요. | 네네. | 어 아까 조금 전에 전화드렸는데 어, 네네 ...",구급 - 질병(중증 외),0


In [6]:
# 전처리 데이터 로드
with open("/content/drive/MyDrive/cleaned_data_preprocess.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [7]:
df = pd.DataFrame(data).copy()

In [8]:
df.head()

,text,disasterType,urgencyLevel
0,예 엊그저께 코로나 주사 맞앗는데 몸이 좀 가슴이 안좋다네요 안면이 마비되고 심장도...,구급 - 질병(중증 외),1
1,아 제가 9일날 월요일날 화이자 백신을 맞았거든요 근데 지금 열도 나고 메슥거리고 ...,구급 - 질병(중증 외),1
2,예 여보세요 저 허리가 너무 아파서 그러는데 어떡해야 저 아까 낮에 병원 잠깐 갔다...,구급 - 질병(중증 외),1
3,예 네 수고하십니다 여기 그 월산동 그 새론 주유소인데요 새론 주유소 GS칼텍스 네...,구급 - 질병(중증 외),0
4,여보세요 네네 어 아까 조금 전에 전화드렸는데 어 네네 저 혹 어 조대병원 좀 가려...,구급 - 질병(중증 외),0


In [12]:
# 임베딩 데이터 로드
with open("/content/drive/MyDrive/embedding.json", "r", encoding="utf-8") as f:
    data_emb = json.load(f)

In [21]:
data = pd.DataFrame(data_emb).copy()

In [22]:
data.head(5)

,embedding,disaster_type,label
0,"[-0.0634055882692337, 0.6826924681663513, 0.00...","[0.0, 0.0, 1.0]",1
1,"[-0.01739795133471489, 0.9349530935287476, 0.0...","[0.0, 0.0, 1.0]",1
2,"[-0.12031228095293045, 0.6935325860977173, 0.2...","[0.0, 0.0, 1.0]",1
3,"[-0.00626472057774663, 0.8518853187561035, 0.0...","[0.0, 0.0, 1.0]",0
4,"[-0.29302048683166504, 0.8401180505752563, -0....","[0.0, 0.0, 1.0]",0


# 데이터 정제

In [ ]:
#모든 파일 로딩
def get_json_files(directories):
    """ 여러 폴더에서 모든 JSON 파일 목록을 가져옵니다. """
    json_files = []
    for directory in directories:
        for f in os.listdir(directory):
            if f.endswith('.json'):
                json_files.append(os.path.join(directory, f))
    return json_files

paths = [
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_광주_구급",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_광주_구조",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_광주_기타",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_광주_화재",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_서울_구급",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_서울_구조",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_서울_기타",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_서울_화재",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_인천_구급",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_인천_구조",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_인천_기타",
    "C:\\Users\\byunddong\\Desktop\\119데이터\\Training\\TL_인천_화재"
]

# 결과 확인
json_files = get_json_files(paths)
print(f" 총 JSON 파일 수: {len(json_files)}개")

In [ ]:
# JSON 파일을 읽어서 정제하고, 결과를 output_file에 저장하는 함수
def clean_data(directories, output_file):
    json_files = get_json_files(directories)
    cleaned_data = []

    for file in json_files:
        with open(file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)

                # 데이터가 리스트인지 아니면 딕셔너리인지 확인
                if isinstance(data, list):
                    # 리스트 형식일 경우
                    for entry in data:
                        user_utterances = [utterance['text'].strip() for utterance in entry['utterances'] if utterance['speaker'] == 1]
                        cleaned_text = ' | '.join(user_utterances)

                        cleaned_entry = {
                            "text": cleaned_text,
                            "disasterType": f"{entry['disasterLarge']} - {entry['disasterMedium']}",
                            "urgencyLevel": 2 if entry['urgencyLevel'] == "상" else 1 if entry['urgencyLevel'] == "중" else 0
                        }
                        cleaned_data.append(cleaned_entry)
                elif isinstance(data, dict):
                    # 딕셔너리 형식일 경우
                    user_utterances = [utterance['text'].strip() for utterance in data['utterances'] if utterance['speaker'] == 1]
                    cleaned_text = ' | '.join(user_utterances)

                    cleaned_entry = {
                        "text": cleaned_text,
                        "disasterType": f"{data['disasterLarge']} - {data['disasterMedium']}",
                        "urgencyLevel": 2 if data['urgencyLevel'] == "상" else 1 if data['urgencyLevel'] == "중" else 0
                    }
                    cleaned_data.append(cleaned_entry)
                else:
                    print(f"경고: {file}의 데이터 형식이 예상과 다릅니다. (리스트나 딕셔너리여야 함)")

            except json.JSONDecodeError:
                print(f"오류: {file} 파일을 읽을 수 없습니다.")

    # 정제된 데이터 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

    print(f"{len(cleaned_data)}개의 데이터가 정제되어 {output_file}에 저장되었습니다!")

clean_data(paths, "cleaned_data.json")


#Function

In [ ]:
# 전처리 함수
def preprocess_text(text):

    # 한국어 불용어 리스트
    stopwords = ["개인정보"]

    # 1. 특수문자 제거
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)

    # 2. 공백 두 개 이상을 하나로 변환
    text = re.sub(r"\s+", " ", text)

    # 3. 반복 단어 제거
    text = re.sub(r"(\b\w+\b)( \1)+", r"\1", text)

    # 4. 불용어 제거
    words = text.split()
    words = [word for word in words if word not in stopwords]

    # 최종 텍스트 반환
    return ' '.join(words)

# Text Preprocessing

In [ ]:
# 1. 원본 정제 Dataframe
original = pd.DataFrame(data).copy()

# 2. 텍스트 전처리 함수 적용
original['text'] = original['text'].apply(preprocess_text)

# 3. 결측치 제거
original = original.dropna(subset=["text", "disasterType", "urgencyLevel"])

# 4. 최종 JSON 저장
original.to_json("/content/drive/MyDrive/cleaned_data_preprocess.json", orient="records", force_ascii=False, indent=4)

print("전처리 + 결측치 제거 완료. 최종 파일 저장됨.")

전처리 + 결측치 제거 완료. 최종 파일 저장됨.


# Data Preprocessing

## Text Embedding

In [ ]:
# 전처리 파일 Dataframe
df = pd.DataFrame(data).copy()

In [ ]:
# 텍스트 임베딩_KcElectra모델

# GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
model = ElectraModel.from_pretrained("beomi/KcELECTRA-base-v2022")
model.to(device)
model.eval()

# 텍스트 임베딩 계산 후 저장할 곳
all_embeddings = []

# 텍스트 임베딩 계산
for text in tqdm(df["text"], desc="임베딩 중"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰의 벡터

    all_embeddings.append(cls_embedding.cpu().squeeze().numpy().tolist())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
임베딩 중: 100%|██████████| 121054/121054 [23:11<00:00, 86.96it/s]


## Disaster Type One Hot Encoding

In [ ]:
# 재난유형 그룹맵핑 및 원핫인코딩

# new_disasterType
disaster_group_mapping = {
    # High_Urgency_Group(긴급도 2가 가장 많음)
    "구급 - 심정지": "High_Urgency_Group",
    "구급 - 약물중독": "High_Urgency_Group",
    "구급 - 임산부": "High_Urgency_Group",
    "구급 - 질병(중증)": "High_Urgency_Group",
    "구조 - 자살": "High_Urgency_Group",
    "화재 - 일반화재": "High_Urgency_Group",
    "화재 - 기타화재": "High_Urgency_Group",
    "화재 - 산불": "High_Urgency_Group",

    # Medium_Urgency_Group(긴급도 1이 가장 많음)
    "구급 - 기타구급": "Medium_Urgency_Group",
    "구급 - 사고": "Medium_Urgency_Group",
    "구급 - 질병(중증 외)": "Medium_Urgency_Group",
    "구조 - 대물사고": "Medium_Urgency_Group",
    "구조 - 안전사고": "Medium_Urgency_Group",

    # Low_Urgency_Group(긴급도 0이 가장 많음)
    "구급 - 부상": "Low_Urgency_Group",
    "구조 - 기타구조": "Low_Urgency_Group",
    "기타 - 기타": "Low_Urgency_Group",
}

df["new_disasterType"] = df["disasterType"].map(disaster_group_mapping).fillna("Unknown")

# new_disasterType 원-핫 인코딩
encoder = OneHotEncoder(sparse_output=False)
disaster_type_encoded = encoder.fit_transform(df[["new_disasterType"]])

In [ ]:
# 긴급도 라벨
all_labels = df["urgencyLevel"]

In [ ]:
#파일 저장
final_results = []

for i in range(len(df)):
    item = {
        "embedding": all_embeddings[i],
        "disaster_type": disaster_type_encoded[i].tolist(),
        "label": int(all_labels[i])
    }
    final_results.append(item)

with open("embedding.json", "w", encoding="utf-8") as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print("JSON 파일 저장 완료: embedding.json")

JSON 파일 저장 완료: embedding.json


## Sampling

In [23]:
# 임베딩 df
# data = pd.DataFrame(data_emb)

In [24]:
# 언더샘플링 전 라벨 분포
print("언더샘플링 전 라벨 분포:\n", data["label"].value_counts())

언더샘플링 전 라벨 분포:
 label
0    45044
1    42835
2    39299
Name: count, dtype: int64


In [25]:
# 언더샘플링
size = 39299
data_0 = data[data["label"] == 0].sample(size, random_state=42)
data_1 = data[data["label"] == 1].sample(size, random_state=42)
data_2 = data[data["label"] == 2].sample(size, random_state=42)

final_df = pd.concat([data_0, data_1, data_2], axis = 0)

print("언더샘플링 후 라벨 분포:\n", final_df["label"].value_counts())

언더샘플링 후 라벨 분포:
 label
0    39299
1    39299
2    39299
Name: count, dtype: int64


## Data Spliting

In [26]:
# 입력과 라벨 정의
X_embed = final_df["embedding"]
X_disaster = final_df[["disaster_type"]]
y = final_df["label"]

In [27]:
# 훈련/검증/테스트 데이터 분할 (8:1:1)
#훈련/나머지 분할
X_embed_train, X_embed_temp, X_disaster_train, X_disaster_temp, y_train, y_temp = train_test_split(X_embed, X_disaster, y, test_size=0.2, stratify=y, random_state=42)
#검증/테스트 분할
X_embed_val, X_embed_test, X_disaster_val, X_disaster_test, y_val, y_test = train_test_split(X_embed_temp, X_disaster_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# 훈련, 검증, 테스트 데이터 라벨 분포 확인
print("훈련 데이터 라벨 분포:\n", y_train.value_counts())
print("검증 데이터 라벨 분포:\n", y_val.value_counts())
print("테스트 데이터 라벨 분포:\n", y_test.value_counts())

훈련 데이터 라벨 분포:
 label
2    31439
1    31439
0    31439
Name: count, dtype: int64
검증 데이터 라벨 분포:
 label
1    3930
2    3930
0    3930
Name: count, dtype: int64
테스트 데이터 라벨 분포:
 label
0    3930
2    3930
1    3930
Name: count, dtype: int64


In [28]:
# 리스트 형태의 벡터를 넘파이 배열로 변환
X_embed_train = np.array(X_embed_train.tolist())
X_embed_val = np.array(X_embed_val.tolist())
X_embed_test = np.array(X_embed_test.tolist())

X_disaster_train = np.array(X_disaster_train["disaster_type"].tolist())
X_disaster_val = np.array(X_disaster_val["disaster_type"].tolist())
X_disaster_test = np.array(X_disaster_test["disaster_type"].tolist())

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Model Training

## Model Defining

In [61]:
text_dim = X_embed_train.shape[1]
disaster_dim = X_disaster_train.shape[1]

def build_mlp_with_dim_reduction(
text_input_dim=768,
reduced_text_dim=16,
disaster_input_dim=3,
hidden1=768, hidden2=384, hidden3=192, hidden4=64,
output_dense=19,
output_dim=3):

# 텍스트 임베딩 입력
    text_input = layers. Input(shape=(text_input_dim,), name="text_input")

    x = layers.Dense(hidden1, kernel_regularizer=regularizers.l2(1e-3))(text_input)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(hidden2, kernel_regularizer=regularizers.l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(hidden3, kernel_regularizer=regularizers.l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(hidden4, kernel_regularizer=regularizers.l2(1e-3))(x)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.4)(x)

# 실제 차원 축소 단계
    x = layers.Dense(reduced_text_dim, activation='relu')(x)

# 재난유형 입력
    disaster_input = layers.Input(shape=(disaster_input_dim,), name="disaster_input")

# concat 이후 추가 은닉층
    concatenated = layers.Concatenate()([x, disaster_input])
    x = layers.Dense(output_dense, activation='relu')(concatenated)
    x = layers.Dropout(0.3)(x)

# 출력층
    outputs = layers.Dense(output_dim, activation='softmax')(x)

    model = models.Model(inputs=[text_input, disaster_input], outputs=outputs)
    return model

## Model Initialization

In [62]:
model = build_mlp_with_dim_reduction(
    text_input_dim=768,
    disaster_input_dim=disaster_dim
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [63]:
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_58 (Dense)    │ (None, 768)       │    590,592 │ text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 768)       │      3,072 │ dense_58[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_34 (ReLU)     │ (None, 768)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_38          │ (None, 768)       │          0 │ re_lu_34[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_59 (Dense)    │ (None, 384)       │    295,296 │ dropout_38[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 384)       │      1,536 │ dense_59[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_35 (ReLU)     │ (None, 384)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_39          │ (None, 384)       │          0 │ re_lu_35[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_60 (Dense)    │ (None, 192)       │     73,920 │ dropout_39[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 192)       │        768 │ dense_60[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_36 (ReLU)     │ (None, 192)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_40          │ (None, 192)       │          0 │ re_lu_36[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_61 (Dense)    │ (None, 64)        │     12,352 │ dropout_40[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64)        │        128 │ dense_61[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_37 (ReLU)     │ (None, 64)        │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_41          │ (None, 64)        │          0 │ re_lu_37[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_62 (Dense)    │ (None, 16)        │      1,040 │ dropout_41[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disaster_input      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_10      │ (None, 19)        │          0 │ dense_62[0][0], 

 Total params: 979,144 (3.74 MB)

 Trainable params: 976,456 (3.72 MB)

 Non-trainable params: 2,688 (10.50 KB)

## Training

In [64]:
# 모델 학습
history = model.fit(
    [X_embed_train, X_disaster_train], y_train,
    validation_data=([X_embed_val, X_disaster_val], y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.4046 - loss: 2.6013 - val_accuracy: 0.5665 - val_loss: 1.9888
Epoch 2/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.5374 - loss: 1.8572 - val_accuracy: 0.5752 - val_loss: 1.4164
Epoch 3/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5502 - loss: 1.3652 - val_accuracy: 0.5759 - val_loss: 1.1416
Epoch 4/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.5585 - loss: 1.1338 - val_accuracy: 0.5732 - val_loss: 1.0183
Epoch 5/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.5600 - loss: 1.0260 - val_accuracy: 0.5724 - val_loss: 0.9597
Epoch 6/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.5615 - loss: 0.9732 - val_accuracy: 0.5707 - val_loss: 0.9327
Epoch 7/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.5650 - loss: 0.9438 - val_accuracy: 0.5774 - val_loss: 0.9085
Epoch 8/100
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5676 -

# Model Evaluation


In [ ]:
y_pred_probs = model.predict([X_embed_test, X_disaster_test])
y_pred = np.argmax(y_pred_probs, axis=1)

In [67]:
# 테스트 평가
y_pred_probs = model.predict([X_embed_test, X_disaster_test])
y_pred = np.argmax(y_pred_probs, axis=1)

print("테스트 성능:")
print(classification_report(y_test, y_pred))

369/369 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
테스트 성능:
              precision    recall  f1-score   support

           0       0.55      0.55      0.55      3930
           1       0.45      0.60      0.52      3930
           2       0.89      0.59      0.71      3930

    accuracy                           0.58     11790
   macro avg       0.63      0.58      0.59     11790
weighted avg       0.63      0.58      0.59     11790

